# Phase 5: Gemini API 기반 인기글 생성

## 📌 목적
Phase 2에서 추출한 **팩터(인기유형)**, **피처(인기요소)**, **키워드** 정보를
Gemini API 프롬프트에 통합하여 서담 스타일 인기글을 생성합니다.

### 핵심 구조
```
팩터(감정적_공감 등) + 피처(실제 인기글 패턴) + 키워드
  → Gemini 프롬프트 → Thinking(전략) → Title + Content
```

### 사전 조건
- `phase2_dataset_v2.ipynb` 실행 완료
- Colab Secrets에 `GEMINI_API_KEY` 등록
- GPU **불필요** (API 기반)

---
## 0. 환경 설정

In [ ]:
from google.colab import drive, userdata
drive.mount('/content/drive', force_remount=True)

import google.generativeai as genai
import json, time, re
from pathlib import Path
from datetime import datetime
import pandas as pd
from IPython.display import display

# ── Gemini API 연결 ─────────────────────────────────────
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)
gemini_model = genai.GenerativeModel('gemini-2.0-flash')

print("✅ Gemini API 연결 완료")

---
## 1. RPM 데이터 로드 (팩터 + 피처 + 키워드)

In [ ]:
PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
OUTPUT_DIR   = PROJECT_ROOT / "outputs"

# ── 팩터 정보 ────────────────────────────────────────────
FACTORS_PATH = OUTPUT_DIR / "rpm_factors.json"
if FACTORS_PATH.exists():
    with open(FACTORS_PATH, "r", encoding="utf-8") as f:
        factors_data = json.load(f)
    factor_names = [f["factor_name"] for f in factors_data["factors"]]
    factor_descriptions = {
        f["factor_name"]: f.get("description", "")
        for f in factors_data["factors"]
    }
    factor_stats = factors_data.get("factor_stats", {})
    print(f"✅ 팩터 로드: {len(factor_names)}개")
    for name in factor_names:
        cov = factor_stats.get(name, {}).get("coverage", 0)
        print(f"  → {name} (커버리지: {cov:.0%}): {factor_descriptions.get(name, '')[:50]}")
else:
    factor_names = ["감정적_공감", "유머_반전", "정보_전달", "참여_유도", "일상_공유", "논란_이슈"]
    factor_descriptions = {
        "감정적_공감": "독자의 감정적 공감을 유발하는 내용이나 어투",
        "유머_반전": "웃음을 유발하는 유머러스한 내용이나 반전 요소",
        "정보_전달": "유용한 정보를 효과적으로 전달하는 구조나 내용",
        "참여_유도": "질문/투표 등으로 독자의 적극적 참여를 유도",
        "일상_공유": "대학생 일상의 보편적 경험을 공유하여 공감 획득",
        "논란_이슈": "의견이 갈리는 주제로 토론을 유발",
    }
    factor_stats = {}
    print("⚠️ 팩터 파일 없음 → 기본 6개 팩터 사용")

# ── 피처 정보 ────────────────────────────────────────────
FEATURES_PATH = OUTPUT_DIR / "rpm_features.json"
if FEATURES_PATH.exists():
    with open(FEATURES_PATH, "r", encoding="utf-8") as f:
        all_features = json.load(f)
    # 의미 있는 피처만 필터링 (기본값 '내용_구성' 제외)
    good_features = []
    for pk, data in all_features.items():
        for feat in data.get("features", []):
            fn = feat.get("feature_name", feat.get("name", ""))
            ctx = feat.get("context", "")
            if fn and fn != "내용_구성" and len(ctx) > 10:
                good_features.append({"name": fn, "context": ctx[:80]})
    print(f"✅ 피처 로드: 전체 {len(all_features)}건 중 의미 있는 피처 {len(good_features)}개")
else:
    all_features = {}
    good_features = []
    print("⚠️ 피처 파일 없음")

# ── 토픽 키워드 ──────────────────────────────────────────
PROMPT_FILES = {
    "익게2":     OUTPUT_DIR / "익게2_topics_for_prompt.json",
    "자유게시판": OUTPUT_DIR / "자유게시판_topics_for_prompt.json",
    "연애상담소": OUTPUT_DIR / "연애상담소_topics_for_prompt.json",
    "익게1":     OUTPUT_DIR / "익게1_topics_for_prompt.json",
}

trend_keywords = {}
for board, path in PROMPT_FILES.items():
    if not path.exists():
        print(f"⚠️ {board}: 파일 없음 → 건너뜀")
        continue
    with open(path, "r", encoding="utf-8") as f:
        topics_data = json.load(f)
    top_topics = sorted(
        topics_data,
        key=lambda x: len(x.get("representatives", [])),
        reverse=True
    )[:5]
    keywords = []
    for topic in top_topics:
        kw = topic.get("keywords", [])
        if isinstance(kw, list):
            keywords.extend(kw[:8])
    trend_keywords[board] = list(dict.fromkeys(keywords))[:8]
    print(f"✅ {board}: {trend_keywords[board]}")

if not trend_keywords:
    print("⚠️ 키워드 파일이 없습니다. 수동 키워드를 사용합니다.")
    trend_keywords = {
        "익게2": ["시험", "교수", "학점", "과제", "MT"],
        "자유게시판": ["취업", "대학원", "졸업", "동아리"],
        "연애상담소": ["남친", "여친", "이별", "고백", "썸"],
    }

---
## 2. Gemini 헬퍼 + 생성 함수

In [ ]:
def call_gemini(prompt: str, temperature: float = 0.7) -> str:
    """Gemini API 호출 (재시도 포함)"""
    for attempt in range(3):
        try:
            response = gemini_model.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=temperature,
                    max_output_tokens=2048,
                ),
            )
            return response.text.strip()
        except Exception as e:
            if "429" in str(e) or "rate" in str(e).lower():
                wait = 15 * (attempt + 1)
                print(f"  ⏳ Rate limit, {wait}초 대기")
                time.sleep(wait)
            else:
                print(f"  ⚠️ 에러: {str(e)[:100]}")
                return ""
    return ""


import random

def generate_post_gemini(
    board_name: str,
    keywords: list,
    factor_name: str,
    factor_desc: str = "",
) -> dict:
    """
    Gemini로 CoT 기반 인기글 생성.
    팩터 + 피처 + 키워드를 프롬프트에 통합.
    """
    # 피처 예시 랜덤 샘플 (매번 다른 조합)
    if good_features:
        sampled = random.sample(good_features, min(8, len(good_features)))
        feature_sample = "\n".join(
            f"  - {f['name']}: {f['context']}"
            for f in sampled
        )
    else:
        feature_sample = "  (피처 데이터 없음)"

    prompt = f"""당신은 서강대학교 커뮤니티 '서담(서강담벼락)'의 인기글 작성 전문가입니다.

아래 조건에 맞춰 인기글을 작성하세요.

[게시판] {board_name}
[키워드] {', '.join(keywords)}
[인기 요인 (팩터)] {factor_name}: {factor_desc}

[참고: 실제 인기글에서 추출된 인기 요소(피처) 예시]
{feature_sample}

위 정보를 바탕으로 다음 형식에 맞춰 작성하세요:

1. <Thinking> 태그: "왜 이런 요소가 인기를 끄는지" + "어떤 전략으로 글을 쓸지" 2~3문장으로 정리
2. <Title> 태그: 클릭을 유발하는 제목 (서담 특유의 말투 사용)
3. <Content> 태그: 실제 게시글 본문

필수 규칙:
- 서담 대학생 커뮤니티 특유의 자연스러운 말투를 사용할 것
- 형식적이거나 AI가 쓴 것 같은 문체 절대 금지
- 실제 서강대생이 쓴 것처럼 느껴져야 함
- 이모티콘이나 특수문자는 자제하되, 자연스러운 경우 허용

<Thinking>
(추론)
</Thinking>
<Title>
(제목)
</Title>
<Content>
(본문)
</Content>
"""

    result = call_gemini(prompt, temperature=0.8)

    # 태그 파싱
    thinking = re.search(r'<Thinking>(.*?)</Thinking>', result, re.DOTALL)
    title    = re.search(r'<Title>(.*?)</Title>', result, re.DOTALL)
    content  = re.search(r'<Content>(.*?)</Content>', result, re.DOTALL)

    return {
        "board_name": board_name,
        "keywords": ", ".join(keywords),
        "factor": factor_name,
        "thinking": thinking.group(1).strip() if thinking else "",
        "generated_title": title.group(1).strip() if title else "",
        "generated_content": content.group(1).strip() if content else result[:500],
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "raw_response": result,
    }


print("✅ CoT 기반 Gemini 생성 함수 정의 완료")

---
## 3. 게시판별 + 팩터별 생성 실행

In [ ]:
generated_posts = []
top_factors = factor_names[:3] if len(factor_names) >= 3 else factor_names

print(f"📋 생성 계획: {len(trend_keywords)}개 게시판 × {len(top_factors)}개 팩터 = {len(trend_keywords) * len(top_factors)}건")
print(f"   팩터: {top_factors}")
print(f"   게시판: {list(trend_keywords.keys())}")
print()

for board, keywords in trend_keywords.items():
    if not keywords:
        continue

    for factor in top_factors:
        desc = factor_descriptions.get(factor, "")
        print(f"🔧 생성 중: [{board}] 팩터={factor}")

        try:
            result = generate_post_gemini(board, keywords, factor, desc)
            generated_posts.append(result)

            print(f"  💭 Thinking: {result['thinking'][:100]}")
            print(f"  📝 제목: {result['generated_title']}")
            print(f"  📄 내용: {result['generated_content'][:150]}...")
            print()

        except Exception as exc:
            print(f"  ⚠️ 생성 실패: {exc}")

        time.sleep(1)

print(f"\n✅ 총 {len(generated_posts)}건 생성 완료")

---
## 4. 결과 저장 및 요약

In [ ]:
if generated_posts:
    result_df = pd.DataFrame(generated_posts)
    # raw_response는 CSV에서 제외
    save_cols = [c for c in result_df.columns if c != "raw_response"]

    output_csv = OUTPUT_DIR / "generated_posts_gemini.csv"
    result_df[save_cols].to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"✅ CSV 저장 완료: {output_csv}")

    output_json = OUTPUT_DIR / "generated_posts_gemini.json"
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(generated_posts, f, ensure_ascii=False, indent=2)
    print(f"✅ JSON 저장 완료: {output_json}")

    print("\n" + "=" * 70)
    print("생성 결과 요약".center(70))
    print("=" * 70)
    display(result_df[["board_name", "factor", "generated_title"]])
else:
    print("⚠️ 생성된 게시글이 없습니다.")

---
## 5. CoT 상세 출력 (Thinking → Response)

In [ ]:
print("\n" + "=" * 70)
print("CoT 상세 출력 (Thinking → Title → Content)".center(70))
print("=" * 70)

for i, post in enumerate(generated_posts, 1):
    print(f"\n{'─' * 70}")
    print(f"[{i}] [{post['board_name']}] 팩터: {post['factor']}")
    print(f"    키워드: {post['keywords']}")
    print(f"\n    💭 Thinking:")
    print(f"    {post['thinking']}")
    print(f"\n    📝 제목: {post['generated_title']}")
    print(f"\n    📄 내용:")
    for line in post['generated_content'].split('\n')[:15]:
        print(f"    {line}")
    if len(post['generated_content'].split('\n')) > 15:
        print(f"    ... (이하 생략)")
    print(f"{'─' * 70}")

---
## 6. 완료

### 생성된 파일
| 파일 | 설명 |
|------|------|
| `generated_posts_gemini.csv` | 생성된 인기글 (표 형태) |
| `generated_posts_gemini.json` | 생성된 인기글 (상세 JSON) |

### 활용 방법
- 생성된 글의 품질을 수작업으로 검토
- `factor_name`과 키워드 조합을 바꿔가며 다양한 스타일 생성 가능
- `temperature`를 0.5~1.0 사이에서 조절하여 창의성 조정